# Setup Inicial

In [19]:
try:
    import unidecode
except ModuleNotFoundError:
    !pip install unidecode

## Bibliotecas utilizadas

In [20]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import json

## Definições gerais

In [21]:
blob_container_name = 'general' # replace with your container name
blob_relative_path_raw = 'raw/mercado_potencial/anp/' # replace with your relative folder path
blob_relative_path_enriched = 'enriched/mercado_potencial/anp/' # replace with your relative folder path
linked_service_raw = 'LS_ADLS_RAW_01' # replace with your linked service name
linked_service_enriched = 'LS_ADLS_ENRICHED_01' # replace with your linked service name


ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

#coletando o endpoint
end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]

#Utilizado na leitura via metodo mssparkutils
abfss_path_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
abfss_path_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)



#dir_enriched = 'abfss://general/enriched/mercado_potencial/anp/'
#ls_enriched = 'LS_ADLS_ENRICHED_01'

## Funções de utilizadas

In [22]:
def ajustar_data(mes, ano):
    """Ajusta e agrupa as datas em formato de ano e mês separados.

        Ajusta as datas que são apresentadas em colunas separadas e em formato string. 
        
    Parâmetros
    ----------
    mes: string
        Lista com o nome do mês reduzido em portugues
    ano: int 
        Lista com os anos
       
    Retornos
    -------
    data_ajustada: datatime
        Lista com data em formato datatime.
    """ 
    
    #dicionário com representação dos meses compactos minusculos em portugues
    meses = {'jan': '01', 'fev': '02', 'mar': '03',
            'abr': '04', 'mai': '05', 'jun': '06',
            'jul': '07', 'ago': '08', 'set': '09',
            'out': '10', 'nov': '11', 'dez': '12'}

    # list comprehension percorre os dados de 'mes' convertendo em minusculo e já 
    # trazendo o valor equivalente do dicionário de meses acima
    mes_minusculo = [meses[i.lower()] for i in mes]

    #cria coluna data a partir do ano e mes
    data_ajustada = ano.astype(str) + mes_minusculo
    
    # converte em formato datatime
    data_ajustada = pd.to_datetime(data_ajustada, format='%Y%m')

    return(data_ajustada)


# Execução

## Leitura dos dados da camada row

In [23]:
arquivo = abfss_path_raw + 'vendas-combustiveis-segmento-m3.csv'
diesel_anp = pd.read_csv( arquivo , sep=";", decimal=',' , storage_options = {'linked_service' : linked_service_raw})

## Ajustes iniciais

In [24]:
# In[4]: PREPARA DADOS DA ANP

# Ajustes dos dados coletados
# remove acentos e deixa nome das colunas em minúsculo;
# mantem somente o produto DIESEL

# Ajusta os titulos sem acento e em minusculo
diesel_anp.columns = [unidecode(i.lower()) for i in list(diesel_anp.columns)]
diesel_anp.rename(columns={'unidade da federacao':'uf'}, inplace=True)

# Elimina os demais combustíveis diferente de diesel
diesel_anp = diesel_anp[diesel_anp['produto'].str.contains('DIESEL')].reset_index(drop=True)

# Melt de vendas por segmento e uf
diesel_uf = diesel_anp.copy()
diesel_uf = diesel_uf.set_index(['ano', 'mes','uf', 'segmento'])['vendas'].unstack().reset_index()

# Ajusta dos nomes das colunas para minúscula
diesel_uf.columns = [unidecode(i.lower()) for i in list(diesel_uf.columns)]

# Agrupa dados por Mês/Ano, Segmento, Brasil
# Agrupa vendas Brasil por mês  
diesel_br = pd.DataFrame(diesel_uf.groupby(['ano','mes']).sum().reset_index())

# Ajusta data e removendo colunas de mes e ano
diesel_br['data'] = ajustar_data(diesel_br['mes'],diesel_br['ano'])
diesel_br.drop(['ano', 'mes'], axis=1, inplace=True)

# Seta data com indice
diesel_br = diesel_br.sort_values(by='data') # ordenando por data
diesel_br = diesel_br.set_index('data', drop=False) # mantendo a data como variavel para ajudar na hora do gráfico

# Remove Posto Revendedor
diesel_br.drop('posto revendedor',axis=1, inplace=True)

## Criação do índice de volume de vendas de diesel

In [25]:
# criando indice com base jan/2012=100

indice = ['Indice_ConsumidorFinal', 'Indice_TRR' ]
diesel_br[indice] = np.nan

for i,c in enumerate(diesel_br.columns[0:2]):
    diesel_br[indice[i]] = diesel_br[c].values / diesel_br[c][0] * 100

## Ajustes finais

In [26]:
#ordenando colunas
diesel_br = diesel_br[diesel_br.columns[[2,0,3,1,4]]]

## Salvando dados mensais da ANP na camada eriched

In [27]:
diesel_br.to_parquet(abfss_path_enriched + 'anp_diesel_mes.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})

# Acesso à documentação

In [28]:
help(ajustar_data)